In [1]:
! mkdir pdfs

In [7]:
from langchain.document_loaders import PyPDFDirectoryLoader

loader = PyPDFDirectoryLoader("pdfs")
data = loader.load()

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
text_chunks = text_splitter.split_documents(data)
print(len(text_chunks))


10


In [9]:
from langchain.embeddings import HuggingFaceHubEmbeddings

embeddings = HuggingFaceHubEmbeddings(model="sentence-transformers/all-MiniLM-L6-v2")

In [11]:
from  langchain.vectorstores import FAISS

VECTORDATABASE = FAISS.from_documents(text_chunks, embeddings)

In [15]:
query = "Explain about correlation"
docs = VECTORDATABASE.similarity_search(query, k=2)
docs

[Document(page_content='Importance\nand\nthe\nuse\nof\ncorrelation\nin\nStatistics\nIntroduction\nCorrelation\nis\na\nstatistical\nmeasure\nthat\nexpresses\nthe\nextent\nto\nwhich\ntwo\nvariables\nare\nlinearly\nrelated.\nIt\nis\na\ncommon\ntool\nfor\ndescribing\nsimple\nrelationships\nwithout\nmaking\na\nstatement\nabout\ncause\nand\neffect.\nCorrelation\ncoefficients\nrange\nfrom\n-1\nto\n1,\nwith\na\nvalue\nof\n0\nindicating\nno\nlinear\nrelationship\nbetween\nthe\ntwo\nvariables,\na\nvalue\nof\n1\nindicating\na\nperfect\npositive\nlinear\nrelationship,\nand\na\nvalue\nof\n-1\nindicating\na', metadata={'source': 'pdfs\\Report - Importance and the use of correlation in Statistics.pdf', 'page': 0}),
 Document(page_content='and\nmental\ndisorders.\nThis\ninformation\ncan\nbe\nused\nto\ndevelop\nbetter\npsychological\nassessments\nand\ntreatments.\nHow\ncorrelation\nis\nused\nin\nthe\nreal\nworld\n1.\nA\nmarketing\nmanager\nmight\nuse\ncorrelation\nto\nidentify\nthe\nrelationship\nbetwe

In [17]:
import os
GEMINI_API_KEY = os.getenv('GOOGLE_API_KEY')

from langchain_google_genai import ChatGoogleGenerativeAI
LLM = ChatGoogleGenerativeAI(model="gemini-pro", convert_system_message_to_human=True)

In [18]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(llm=LLM, chain_type="stuff", retriever=VECTORDATABASE.as_retriever())

In [19]:
query = 'What is correlation ?'
output = qa.run(query)
print(output)

Correlation is a statistical measure that expresses the extent to which two variables are linearly related. It is a common tool for describing simple relationships without making a statement about cause and effect.
